In [ ]:
import json
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, lpSum
import math

# Function to calculate Euclidean distance between two points
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Load NJ counties and adjacent counties into a dict
njc_file = 'nj_counties.json'
with open(njc_file) as f:
    njc_dict = json.load(f)

total_pop = sum(county_data['population'] for county_data in njc_dict.values())
print("Total population:", total_pop)

# Number of congressional districts
n_districts = 12

# Calculate desired population for each district for population balance
desired_pop = total_pop / n_districts
print('Desired population: ', desired_pop)

# Preprocess counties based on desired population
model_cities = {}
model_n_counties = 0
model_total_pop = 0
model_n_districts = n_districts
print('Counties not used in the model:')
for county, data in njc_dict.items():
    c_pop = data['population']
    if c_pop > desired_pop:
        print('\t', county, c_pop)
        model_n_districts -= 1
    else:
        model_cities[county] = data
        model_n_counties += 1
        model_total_pop += c_pop

print('Number of counties used in the model: ', model_n_counties)
print('Updated total population: ', model_total_pop)
print('Updated number of districts: ', model_n_districts)
print('Updated desired population: ', model_total_pop / model_n_districts)

# Create the model
model = LpProblem("Redistricting", LpMinimize)

# Decision Variables
DV_variable_d = LpVariable.dicts("d", [(i, j) for i in range(model_n_counties) for j in range(model_n_districts)], cat="Binary")

# Calculate distances between all pairs of counties
county_list = list(model_cities.keys())
distance_matrix = {}
for i, county_i in enumerate(county_list):
    for j, county_j in enumerate(county_list):
        if i != j:
            coord_i = model_cities[county_i]['geo_coords']
            coord_j = model_cities[county_j]['geo_coords']
            distance_matrix[(i, j)] = calculate_distance(coord_i, coord_j)

# Constraints
# Ensure each county is assigned to exactly one district
for i in range(model_n_counties):
    model += lpSum(DV_variable_d[(i, j)] for j in range(model_n_districts)) == 1, f"County_{i}_Assignment"

# Ensure districts have populations within desired ranges
for j in range(model_n_districts):
    model += lpSum(DV_variable_d[(i, j)] * model_cities[county_list[i]]['population'] for i in range(model_n_counties)) >= 0.9 * desired_pop, f"District_{j+1}_MinPop"
    model += lpSum(DV_variable_d[(i, j)] * model_cities[county_list[i]]['population'] for i in range(model_n_counties)) <= 1.1 * desired_pop, f"District_{j+1}_MaxPop"

# Objective Function: Minimize the sum of distances indirectly through adjacency considerations
model += lpSum(DV_variable_d[(i, j)] * distance_matrix.get((i, k), 0)
               for i in range(model_n_counties)
               for j in range(model_n_districts)
               for k in range(model_n_counties) if i != k), "Compactness"

# Solve the model
status = model.solve()

# Check the status of the solution
print("Status:", LpProblem.status[status])

# To display the assignments (optional)
for v in model.variables():
    if v.varValue == 1:
        print(v.name, "=", v.varValue)


Total population: 9249175
Desired population:  770764.5833333334
Counties not used in the model:
	 Bergen County 949233
	 Essex County 840189
	 Middlesex County 859598
Number of counties used in the model:  18
Updated total population:  6600155
Updated number of districts:  9
Updated desired population:  733350.5555555555
